- ## Disease Idenfier

In [1]:
import spacy
from tqdm import tqdm
import spacy
import random
import json
import re

In [2]:
training_data = []
lines=[]
with open('Disease Identifier.json', 'r',encoding="utf8") as f:
    lines = f.readlines()

for line in lines:
    data = json.loads(line)
    text = data['content']
    entities = []
    if(data['annotation'] is not None):
        for annotation in data['annotation']:
              if(annotation is not None):
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    #dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                    entities.append((point['start'], point['end'] + 1 ,label))

    if(entities != []):
        training_data.append((text, {"entities" : entities}))

In [3]:
len(training_data)

70

In [4]:
training_data[4:10]

[('Hi Mr Avnish Chaurasia of age 23 years.You have the problems of Unexpected weight loss,night sweats.You are having the disease Congenital Heart Disease.According to report What i found is that you are suffering from Tuberculosis.The prescribed medicine you have to take are chloramphenicol Oily suspension for injection*: 0.5 g (as sodiumsuccinate)/ mL in 2‐ mL ampoule,erythromycin Powder for oral liquid: 125 mg/5 mL (as stearate or estolate or ethyl succinate).You should flex your ankles every half hour or so while sitting.You should try Avoiding foods that trigger indigestion.Drink Warm water.',
  {'entities': [(127, 151, 'Disease')]}),
 ('hello,Mahesh Rathore,You should try Chewing food slowly and thoroughly. You are 25 years old.Try Reducing or eliminating the use of alcohol, caffeine, and carbonated beverages.According to your problems i have read you are having the problems of loose stools,dry Cough.You are suffering from Measles.I am writing the medicines you have to take are c

In [5]:
## Training a blank spacy english model.

nlp = spacy.blank('en')  # create blank Language class
# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)

# add labels
for _, annotations in tqdm(training_data):
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(20):
        print("Statring iteration " + str(itn))
        random.shuffle(training_data)
        losses = {}
        for text, annotations in training_data:
            nlp.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.2,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print(losses)

100%|██████████| 70/70 [00:00<00:00, 43528.73it/s]


Statring iteration 0
{'ner': 578.3663458628191}
Statring iteration 1
{'ner': 1002.5856584489034}
Statring iteration 2
{'ner': 1759.5136606996464}
Statring iteration 3
{'ner': 93.37986619397002}
Statring iteration 4
{'ner': 68.95422435931224}
Statring iteration 5
{'ner': 154.95998495088014}
Statring iteration 6
{'ner': 135.56074940329339}
Statring iteration 7
{'ner': 47.72477854134226}
Statring iteration 8
{'ner': 66.82497233759759}
Statring iteration 9
{'ner': 37.91689513402854}
Statring iteration 10
{'ner': 27.12119677987184}
Statring iteration 11
{'ner': 29.480055596507807}
Statring iteration 12
{'ner': 26.993614311598616}
Statring iteration 13
{'ner': 19.296016337054162}
Statring iteration 14
{'ner': 83.46708708557833}
Statring iteration 15
{'ner': 26.011229229336845}
Statring iteration 16
{'ner': 51.31065226055111}
Statring iteration 17
{'ner': 103.22975881620152}
Statring iteration 18
{'ner': 37.15102032451509}
Statring iteration 19
{'ner': 30.99860058136513}


In [6]:
nlp.to_disk("new_model_2")

In [7]:
nlp5=spacy.load("new_model_2")

In [49]:
# doc=nlp5(open("./MEDICAL_REPORTS/TEST_REPORT_14.txt").read())
doc=nlp5(open("listfile.txt").read())

In [50]:
print(doc)

suman kumar of age 30 years is having these symptoms of dry cough for last three days. but no fever and having running nose the diagnosis report says that the patient have disease acute bronchites and so i am prescribing the following medicines azithromycin 500mg once a day for three days, robitussin 5 ml thrice a day for 5 days and You should drink warm water and dont eat grapes. 




In [51]:
for entity in doc.ents:
    print(entity.text)

disease acute bronchites and so i am prescribing the following medicines azithromycin 500mg once a day for three days, robitussin 5 ml thrice a day for 5 days and You should drink warm water and dont eat grapes. 


